## Workgroup 6

Members:
* Diego Gómez
* Alexander Pacheco

## 2. Debiased Machine Learning

In this section, will use the DML algorithm we have learned. You can use the R and Python scripts from the labs as guides.
For this task you will use the data from The Testing Convergence Hypothesis Lab (Barro-Lee growth data).

In [1]:
# Import relevant packages
import pandas as pd
import numpy as np
import pyreadr
from sklearn import preprocessing
import patsy

from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense

import hdmpy
import numpy as np
import random
import statsmodels.api as sm
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import colors

In [2]:
from sklearn.linear_model import LassoCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import RidgeCV, ElasticNetCV
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
import itertools
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
from pandas.api.types import is_categorical_dtype
from itertools import compress
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.feature_selection import SelectFromModel

# Debiased Machine Learning (Python script)

We provide an additional empirical example of partialling-out with Lasso to estimate the regression coefficient $\beta_1$ in the high-dimensional linear regression model:

   Y = $\beta_1$ D +  $\beta_2$'W + $\epsilon$

Specifically, we are interested in how the rates at which economies of different countries grow ($Y$) are related to the initial wealth levels in each country ($D$) controlling for country's institutional, educational, and other similar characteristics ($W$).

The relationship is captured by $\beta_1$, the speed of convergence/divergence, which measures the speed at which poor countries catch up $(\beta_1< 0)$ or fall behind $(\beta_1> 0)$ rich countries, after controlling for $W$. Our inference question here is: do poor countries grow faster than rich countries, controlling for educational and other characteristics? In other words, is the speed of convergence negative: $ \beta_1 <0?$ This is the Convergence Hypothesis predicted by the Solow Growth Model. This is a structural economic model. Under some strong assumptions, that we won't state here, the predictive exercise we are doing here can be given causal interpretation.

The outcome $Y$ is the realized annual growth rate of a country's wealth (Gross Domestic Product per capita). The target regressor ($D$) is the initial level of the country's wealth. The target parameter $\beta_1$ is the speed of convergence, which measures the speed at which poor countries catch up with rich countries. The controls ($W$) include measures of education levels, quality of institutions, trade openness, and political stability in the country.

## Data analysis

We use the data set GrowthData which is included in the package hdm.

In [3]:
# We downloaded the data that the author used
growth_read = pyreadr.read_r("../data/GrowthData.RData")

# Extracting the data frame from rdata_read
growth = growth_read[ 'GrowthData' ]
list(growth)

['Outcome',
 'intercept',
 'gdpsh465',
 'bmp1l',
 'freeop',
 'freetar',
 'h65',
 'hm65',
 'hf65',
 'p65',
 'pm65',
 'pf65',
 's65',
 'sm65',
 'sf65',
 'fert65',
 'mort65',
 'lifee065',
 'gpop1',
 'fert1',
 'mort1',
 'invsh41',
 'geetot1',
 'geerec1',
 'gde1',
 'govwb1',
 'govsh41',
 'gvxdxe41',
 'high65',
 'highm65',
 'highf65',
 'highc65',
 'highcm65',
 'highcf65',
 'human65',
 'humanm65',
 'humanf65',
 'hyr65',
 'hyrm65',
 'hyrf65',
 'no65',
 'nom65',
 'nof65',
 'pinstab1',
 'pop65',
 'worker65',
 'pop1565',
 'pop6565',
 'sec65',
 'secm65',
 'secf65',
 'secc65',
 'seccm65',
 'seccf65',
 'syr65',
 'syrm65',
 'syrf65',
 'teapri65',
 'teasec65',
 'ex1',
 'im1',
 'xr65',
 'tot1']

In [4]:
growth.shape

(90, 63)

In [5]:
growth.head()

,Outcome,intercept,gdpsh465,bmp1l,freeop,freetar,h65,hm65,hf65,p65,...,seccf65,syr65,syrm65,syrf65,teapri65,teasec65,ex1,im1,xr65,tot1
0,-0.024336,1,6.591674,0.2837,0.153491,0.043888,0.007,0.013,0.001,0.29,...,0.04,0.033,0.057,0.010,47.6,17.3,0.0729,0.0667,0.348,-0.014727
1,0.100473,1,6.829794,0.6141,0.313509,0.061827,0.019,0.032,0.007,0.91,...,0.64,0.173,0.274,0.067,57.1,18.0,0.0940,0.1438,0.525,0.005750
2,0.067051,1,8.895082,0.0000,0.204244,0.009186,0.260,0.325,0.201,1.00,...,18.14,2.573,2.478,2.667,26.5,20.7,0.1741,0.1750,1.082,-0.010040
3,0.064089,1,7.565275,0.1997,0.248714,0.036270,0.061,0.070,0.051,1.00,...,2.63,0.438,0.453,0.424,27.8,22.7,0.1265,0.1496,6.625,-0.002195
4,0.027930,1,7.162397,0.1740,0.299252,0.037367,0.017,0.027,0.007,0.82,...,2.11,0.257,0.287,0.229,34.5,17.6,0.1211,0.1308,2.500,0.003283


### Preprocessing

In [6]:
import re

In [7]:
#################################  Find Variable Names from Dataset ########################

def varlist( df = None, type_dataframe = [ 'numeric' , 'categorical' , 'string' ],  pattern = "", exclude = None ):
    varrs = []
    
    if ('numeric' in type_dataframe):
        varrs = varrs + df.columns[ df.apply( is_numeric_dtype , axis = 0 ).to_list() ].tolist()
    
    if ('categorical' in type_dataframe):
        varrs = varrs + df.columns[ df.apply( is_categorical_dtype , axis = 0 ).to_list() ].tolist()
    
    if ('string' in type_dataframe):
        varrs = varrs + df.columns[ df.apply( is_string_dtype , axis = 0 ).to_list() ].tolist()
    
    grepl_result = np.array( [ re.search( pattern , variable ) is not None for variable in df.columns.to_list() ] )
    
    if exclude is None:
        result = list(compress( varrs, grepl_result ) )
    
    else:
        varrs_excluded = np.array( [var in exclude for var in varrs ] )
        and_filter = np.logical_and( ~varrs_excluded ,  grepl_result )
        result = list(compress( varrs, and_filter ) )
    
    return result   


We now define some variables: n, the treatment variable, the outcome variable and the matrix $Z$ that includes the control variables (country characterisitics)

In [8]:
# Number of observations
n = growth.shape[0]

# Treatment Variable
d = "gdpsh465"   

# Outcome Variable
y = "Outcome" 

# Treatment Variable
D = growth[ f'{d}']

# Outcome Variable
Y = growth[ f'{y}']

# Controls matrix Z
Z = growth.drop(['Outcome','intercept', 'gdpsh465'], 1 )

Z.shape[1]

C:\Users\Hp\.conda\envs\r-tutorial\lib\site-packages\ipykernel_launcher.py:17: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only


60

We notice that we have 60 control variables.

Now we define our data

In [9]:
data = pd.concat([Y, D, Z], axis=1)

## 1. OLS without including the country characteristics

In [10]:
model = "Outcome ~ gdpsh465"
baseline_ols = smf.ols(model , data=data).fit()
baseline_ols_table = baseline_ols.summary2().tables[1]
print( baseline_ols_table.iloc[ 1 , 4:] )
baseline_ols_table.iloc[1, :]

[0.025   -0.010810
0.975]    0.013444
Name: gdpsh465, dtype: float64


Coef.       0.001317
Std.Err.    0.006102
t           0.215777
P>|t|       0.829661
[0.025     -0.010810
0.975]      0.013444
Name: gdpsh465, dtype: float64

The estimated coefficient is $0.001317$ with the confidence interval ranging from -0.01 to 0.01 and it seems not to be significant without controlling for X (P>|t| = 0.829661)

## 2. OLS including the country characteristics

In [11]:
# define the variables
y = 'Outcome'

data_columns = list(data)
no_relev_col = ['Outocome', 'gdpsh465']

z = [col for col in data_columns if col not in no_relev_col] 

In [12]:
control_formula = "Outcome" + ' ~ ' + 'gdpsh465 + ' + ' + '.join( Z.columns.to_list() )

In [13]:
control_ols = smf.ols( control_formula , data=data).fit()
control_ols_table = control_ols.summary2().tables[1]
print( control_ols_table.iloc[ 1 , 4:] )
control_ols_table.iloc[1, :]

[0.025   -0.070600
0.975]    0.051844
Name: gdpsh465, dtype: float64


Coef.      -0.009378
Std.Err.    0.029888
t          -0.313774
P>|t|       0.756019
[0.025     -0.070600
0.975]      0.051844
Name: gdpsh465, dtype: float64

After controlling with country characteristics, we find now a negative coefficient $-0.009378$ but it is also no significant. 

# DML algorithm

Here we perform inference of the predictive coefficient $\beta$ in our partially linear statistical model, 

$$
Y = D\beta + g(Z) + \epsilon, \quad E (\epsilon | D, Z) = 0,
$$

using the **double machine learning** approach. 

For $\tilde Y = Y- E(Y|Z)$ and $\tilde D= D- E(D|Z)$, we can write
$$
\tilde Y = \alpha \tilde D + \epsilon, \quad E (\epsilon |\tilde D) =0.
$$

Using cross-fitting, we employ modern regression methods
to build estimators $\hat \ell(Z)$ and $\hat m(Z)$ of $\ell(Z):=E(Y|Z)$ and $m(Z):=E(D|Z)$ to obtain the estimates of the residualized quantities:

$$
\tilde Y_i = Y_i  - \hat \ell (Z_i),   \quad \tilde D_i = D_i - \hat m(Z_i), \quad \text{ for each } i = 1,\dots,n.
$$

Finally, using ordinary least squares of $\tilde Y_i$ on $\tilde D_i$, we obtain the 
estimate of $\beta$.

The following algorithm comsumes $Y, D, Z$, and a machine learning method for learning the residuals $\tilde Y$ and $\tilde D$, where the residuals are obtained by cross-validation (cross-fitting). Then, it prints the estimated coefficient $\beta$ and the corresponding standard error from the final OLS regression.

In [14]:
def DML2_for_PLM(z, d, y, dreg, yreg, nfold):
    
    # Num ob observations
    nobs = z.shape[0]
    
    # Define folds indices 
    list_1 = [*range(0, nfold, 1)]*nobs
    sample = np.random.choice(nobs,nobs, replace=False).tolist()
    foldid = [list_1[index] for index in sample]

    # Create split function(similar to R)
    def split(x, f):
        count = max(f) + 1
        return tuple( list(itertools.compress(x, (el == i for el in f))) for i in range(count) ) 

    # Split observation indices into folds 
    list_2 = [*range(0, nobs, 1)]
    I = split(list_2, foldid)
    
    # Create array to save errors 
    dtil = np.zeros( len(z) ).reshape( len(z) , 1 )
    ytil = np.zeros( len(z) ).reshape( len(z) , 1 )
    
    # loop to save results
    for b in range(0,len(I)):
    
        # Split data - index to keep are in mask as booleans
        include_idx = set(I[b])  #Here should go I[b] Set is more efficient, but doesn't reorder your elements if that is desireable
        mask = np.array([(i in include_idx) for i in range(len(z))])

        # Lasso regression, excluding folds selected 
        dfit = dreg(z[~mask,], d[~mask,])
        yfit = yreg(z[~mask,], y[~mask,])

        # predict estimates using the 
        dhat = dfit.predict( z[mask,] )
        yhat = yfit.predict( z[mask,] )

        # save errors  
        dtil[mask] =  d[mask,] - dhat.reshape( len(I[b]) , 1 )
        ytil[mask] = y[mask,] - yhat.reshape( len(I[b]) , 1 )
        print(b, " ")
    
    # Create dataframe 
    data_2 = pd.DataFrame(np.concatenate( ( ytil, dtil ), axis = 1), columns = ['ytil','dtil'])
   
    model = "ytil ~ dtil"
    baseline_ols = smf.ols(model , data = data_2 ).fit()
    coef_est = baseline_ols.summary2().tables[1]['Coef.']['dtil']
    se = baseline_ols.summary2().tables[1]['Std.Err.']['dtil']
    
    Final_result = { 'coef_est' : coef_est , 'se' : se , 'dtil' : dtil , 'ytil' : ytil }

    print("Coefficient is {}, SE is equal to {}".format(coef_est, se))
    
    return Final_result

Let us, construct the input matrices.

In [15]:
# Create main variables
Y = data['Outcome']
D = data['gdpsh465']
Z = data.drop(['Outcome', 'gdpsh465'], axis=1)

In [16]:
Y.shape

(90,)

In [17]:
D.shape

(90,)

In [18]:
Z.shape

(90, 60)

In [19]:
# as matrix
y = Y.to_numpy().reshape( len(Y) , 1 )
d = D.to_numpy().reshape( len(Y) , 1 )
z = Z.to_numpy()

In the following, we apply the DML approach with the differnt versions of lasso.

## 3. DML using Lasso to predict y and d

In [20]:
def dreg(z,d):
    alpha=0.00000001
    result = linear_model.Lasso(alpha = alpha).fit(z, d)
    return result

def yreg(z,y):
    alpha=0.00000001
    result = linear_model.Lasso(alpha = alpha).fit(z, y)
    return result

DML2_lasso = DML2_for_PLM(z, d, y, dreg, yreg, 10)

C:\Users\Hp\.conda\envs\r-tutorial\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:532: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.7778654724637054, tolerance: 0.006216004653469967
  positive)
C:\Users\Hp\.conda\envs\r-tutorial\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:532: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.01391384759414357, tolerance: 2.1959795148727455e-05
  positive)
C:\Users\Hp\.conda\envs\r-tutorial\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:532: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.9055924402765264, tolerance: 0.006643320002424726
  positive)
C:\Users\Hp\.conda\envs\r-tutorial\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:532: ConvergenceWarning: Objective did not converge. You mig

0  
1  
2  
3  
4  
5  
6  
7  
8  
9  
Coefficient is 0.01854906811333555, SE is equal to 0.010472091220130372


C:\Users\Hp\.conda\envs\r-tutorial\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:532: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.012383961392374365, tolerance: 2.001665167647239e-05
  positive)
C:\Users\Hp\.conda\envs\r-tutorial\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:532: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.7443438793407297, tolerance: 0.006720580888482781
  positive)
C:\Users\Hp\.conda\envs\r-tutorial\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:532: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.013111002981625569, tolerance: 2.1713417963181847e-05
  positive)
C:\Users\Hp\.conda\envs\r-tutorial\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:532: ConvergenceWarning: Objective did not converge. You

## 4. DML using Post-Lasso to predict y and d

In [21]:
class Lasso_post:
    
    def __init__(self, alpha ):
        self.alpha = alpha

        
    def fit( self, X, Y ):
        self.X = X
        self.Y = Y
        lasso = linear_model.Lasso( alpha = self.alpha ).fit( X , Y )
        model = SelectFromModel( lasso , prefit = True )
        X_new = model.transform( X )
        # Gettin indices from columns which has variance for regression
        index_X = model.get_support()
        
        self.index = index_X
        new_x = X[ : ,  index_X ]
        
        lasso2 = linear_model.Lasso( alpha = self.alpha ).fit( new_x , Y )
        self.model = lasso2
        
        return self
    
    def predict( self , X ):
        
        dropped_X = X[ : , self.index ]
        
        predictions = self.model.predict( dropped_X )
        
        return predictions

In [22]:
def dreg(z,d):
    alpha=0.00000001
    result = Lasso_post( alpha = alpha ).fit( z , d )
    return result

def yreg( z , y ):
    alpha = 0.00000001
    result = Lasso_post( alpha = alpha ).fit( z , y )
    return result

DML2_lasso_post = DML2_for_PLM(z, d, y, dreg, yreg, 10)

C:\Users\Hp\.conda\envs\r-tutorial\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:532: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.9488262887411388, tolerance: 0.006664271160822951
  positive)
C:\Users\Hp\.conda\envs\r-tutorial\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:532: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.9709080550705614, tolerance: 0.006664271160822951
  positive)
C:\Users\Hp\.conda\envs\r-tutorial\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:532: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.013405826767078004, tolerance: 2.0850857836082115e-05
  positive)
C:\Users\Hp\.conda\envs\r-tutorial\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:532: ConvergenceWarning: Objective did not converge. You mi

0  
1  
2  
3  


C:\Users\Hp\.conda\envs\r-tutorial\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:532: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.01512597733311275, tolerance: 2.3267649795873873e-05
  positive)
C:\Users\Hp\.conda\envs\r-tutorial\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:532: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.8709141967812717, tolerance: 0.006131709440067548
  positive)
C:\Users\Hp\.conda\envs\r-tutorial\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:532: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.8722102689585127, tolerance: 0.006131709440067548
  positive)
C:\Users\Hp\.conda\envs\r-tutorial\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:532: ConvergenceWarning: Objective did not converge. You mig

4  
5  
6  
7  
8  
9  
Coefficient is 0.01676965127907394, SE is equal to 0.013054251609344222


C:\Users\Hp\.conda\envs\r-tutorial\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:532: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.864967972336677, tolerance: 0.0064309750710534
  positive)
C:\Users\Hp\.conda\envs\r-tutorial\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:532: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.012020658215445477, tolerance: 2.0801225369550993e-05
  positive)
C:\Users\Hp\.conda\envs\r-tutorial\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:532: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.012591538597985707, tolerance: 2.0801225369550993e-05
  positive)
C:\Users\Hp\.conda\envs\r-tutorial\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:532: ConvergenceWarning: Objective did not converge. You m

## 5. DML using Random Forest to predict y and d

In [23]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

In [24]:
z.shape

(90, 60)

In [25]:
#DML with Random Forest:
def dreg(z,d):
    result = RandomForestRegressor( random_state = 0 , n_estimators = 500 , max_features = 60 , n_jobs = 4 , min_samples_leaf = 5 ).fit( z, d )
    return result

def yreg(z,y):
    result = RandomForestRegressor( random_state = 0 , n_estimators = 500 , max_features = 60 , n_jobs = 4 , min_samples_leaf = 5 ).fit( z, y )
    return result

DML2_RF = DML2_for_PLM(z, d, y, dreg, yreg, 2)   # set to 2 due to computation time

C:\Users\Hp\.conda\envs\r-tutorial\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Hp\.conda\envs\r-tutorial\lib\site-packages\ipykernel_launcher.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  import sys


0  


C:\Users\Hp\.conda\envs\r-tutorial\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Hp\.conda\envs\r-tutorial\lib\site-packages\ipykernel_launcher.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  import sys


1  
Coefficient is -0.028341543504056824, SE is equal to 0.011312729595461932


## 6. Run the best method i.e. the best combination of methods to predict y and d.

We define the DML_ols

In [26]:
#DML with OLS:
def dreg(z,d):
    result = LinearRegression().fit( z, d )
    return result

def yreg(z,y):
    result = LinearRegression().fit( z, y )
    return result

DML2_ols = DML2_for_PLM(z, d, y, dreg, yreg, 2)

0  
1  
Coefficient is -0.09201383622019393, SE is equal to 0.012557869958406437


To be able to find the best method, we will find RMSE for predicting D and Y, and see which of the methods works better

In [27]:
mods = [DML2_ols, DML2_lasso, DML2_lasso_post, DML2_RF]
mods_name = ["DML2_ols", "DML2_lasso", "DML2_lasso_post", 'DML2_RF']

def mdl( model , model_name ):
    
    RMSEY = np.sqrt( np.mean( model[ 'ytil' ] ) ** 2 ) # I have some doubts about these equations...we have to recheck
    RMSED = np.sqrt( np.mean( model[ 'dtil' ] ) ** 2 ) # I have some doubts about these equations...we have to recheck
    
    result = pd.DataFrame( { model_name : [ RMSEY , RMSED ]} , index = [ 'RMSEY' , 'RMSED' ])
    return result

RES = [ mdl( model , name ) for model, name in zip( mods , mods_name ) ]
    

pr_Res = pd.concat( RES, axis = 1)

pr_Res

,DML2_ols,DML2_lasso,DML2_lasso_post,DML2_RF
RMSEY,0.092477,0.001877,0.002521,0.001061
RMSED,0.139480,0.019442,0.000181,0.024032


To verifie that the function has no errors:

In [28]:
np.where(DML2_lasso_post[ 'ytil' ] == 0)[0].size

0

The table above indicates that the best method for predicting Y is Random Forest and for predicting D is Post-Lasso. We can deduce this by looking at the RMSE that we find when predicting both Y and D.

In [29]:
#DML with the BEST:
def dreg(z,d):
    alpha=0.00000001
    result = Lasso_post( alpha = alpha ).fit( z , d )
    return result

def yreg(z,y):
    result = RandomForestRegressor( random_state = 0 , n_estimators = 500 , max_features = 60 , n_jobs = 4 , min_samples_leaf = 5 ).fit( z, y )
    return result

DML2_best = DML2_for_PLM(z, d, y, dreg, yreg, 10)

C:\Users\Hp\.conda\envs\r-tutorial\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:532: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.9140421434122824, tolerance: 0.006184685815434688
  positive)
C:\Users\Hp\.conda\envs\r-tutorial\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:532: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.9185506500339666, tolerance: 0.006184685815434688
  positive)
C:\Users\Hp\.conda\envs\r-tutorial\lib\site-packages\ipykernel_launcher.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


0  


C:\Users\Hp\.conda\envs\r-tutorial\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:532: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.7805636139905683, tolerance: 0.006360018491833337
  positive)
C:\Users\Hp\.conda\envs\r-tutorial\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:532: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.7883316158313166, tolerance: 0.006360018491833337
  positive)
C:\Users\Hp\.conda\envs\r-tutorial\lib\site-packages\ipykernel_launcher.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


1  


C:\Users\Hp\.conda\envs\r-tutorial\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:532: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.6979816826173604, tolerance: 0.006278816399322124
  positive)
C:\Users\Hp\.conda\envs\r-tutorial\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:532: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.7079061015044291, tolerance: 0.006278816399322124
  positive)
C:\Users\Hp\.conda\envs\r-tutorial\lib\site-packages\ipykernel_launcher.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


...

7  


C:\Users\Hp\.conda\envs\r-tutorial\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:532: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.9122755368943093, tolerance: 0.006443867699468521
  positive)
C:\Users\Hp\.conda\envs\r-tutorial\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:532: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.9312012955005959, tolerance: 0.006443867699468521
  positive)
C:\Users\Hp\.conda\envs\r-tutorial\lib\site-packages\ipykernel_launcher.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


8  


C:\Users\Hp\.conda\envs\r-tutorial\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:532: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.7220964702820466, tolerance: 0.006359042560623478
  positive)
C:\Users\Hp\.conda\envs\r-tutorial\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:532: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.7446474902587041, tolerance: 0.006359042560623478
  positive)
C:\Users\Hp\.conda\envs\r-tutorial\lib\site-packages\ipykernel_launcher.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


9  
Coefficient is -0.00022232796994351068, SE is equal to 0.011012257199274032


## 7. Show your results in a table as we did in the lab.

Now, we can show our results in the next table.

In [30]:
table = np.zeros( ( 6 , 2 ))
table[ 0 , 0] = baseline_ols_table.iloc[ 1 , 0 ]
table[ 1 , 0] = control_ols_table.iloc[ 1 , 0 ]
table[ 2 , 0] = DML2_lasso['coef_est']
table[ 3 , 0] = DML2_lasso_post['coef_est']
table[ 4 , 0] = DML2_RF['coef_est']
table[ 5 , 0] = DML2_best['coef_est']
table[ 0 , 1] = baseline_ols_table.iloc[ 1 , 1 ]
table[ 1 , 1] = control_ols_table.iloc[ 1 , 1 ]
table[ 2 , 1] = DML2_lasso['se']
table[ 3 , 1] = DML2_lasso_post['se']
table[ 4 , 1] = DML2_RF['se']
table[ 5 , 1] = DML2_best['se']

In [33]:
table = pd.DataFrame( table , index = [ "Baseline OLS", "Least Squares with controls", "Lasso", \
             "Post-Lasso", "Random Forest", "Best" ] , \
            columns = ["Estimate","Standard Error"] )
table.round( 5 )

,Estimate,Standard Error
Baseline OLS,0.00132,0.00610
Least Squares with controls,-0.00938,0.02989
Lasso,0.01855,0.01047
Post-Lasso,0.01677,0.01305
Random Forest,-0.02834,0.01131
Best,-0.00022,0.01101
